In [1]:
# we install all the required for RelBench
!pip install relbench[full]

In [2]:
import relbench
from relbench.datasets import get_dataset_names, get_dataset
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph


import torch
from torch_geometric.seed import seed_everything
from torch import Tensor
from torch_frame.config.text_embedder import TextEmbedderConfig


from typing import List, Optional


from sentence_transformers import SentenceTransformer


import os


import pandas as pd


import random

In [3]:
class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return self.model.encode(sentences, convert_to_tensor=True)

# Environment Checking

In [4]:
# Check that it's cuda if you want it to run in reasonable time!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_num_threads(1)
print(device)

# Set the seed for generating random numbers to ensure reproducibility
seed_everything(42)

# Path to the directory for caching graph data
root_dir = "./data"

# Configure the text encoder
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device),
    batch_size=256
)

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
print(f"The RelBench version is {relbench.__version__}")
print(f"The RelBench datasets are {get_dataset_names()}")

The RelBench version is 1.1.0
The RelBench datasets are ['rel-amazon', 'rel-avito', 'rel-event', 'rel-f1', 'rel-hm', 'rel-stack', 'rel-trial']


#Usefull functions

In [15]:
import random

def remove_proportion(hetero_data, proportion):
    if not (0 <= proportion <= 1):
        raise ValueError("La proporzione deve essere un valore tra 0 e 1.")

    # Funzione per rimuovere una proporzione di dati
    def remove_from_tensor_frame(tensor_frame, proportion):
        total_rows = tensor_frame.tf.shape[0]
        rows_to_remove = int(total_rows * proportion)

        # Seleziona casualmente le righe da rimuovere
        indices_to_remove = random.sample(range(total_rows), rows_to_remove)
        mask = [i for i in range(total_rows) if i not in indices_to_remove]

        # Ritorna un nuovo TensorFrame con le righe rimanenti
        return tensor_frame[mask]

    # Rimuovi dai nodi
    for key in hetero_data.keys():
        if 'edge_index' not in key:  # Ignora gli edge
            hetero_data[key]['tf'] = remove_from_tensor_frame(hetero_data[key]['tf'], proportion)

    # Rimuovi dagli edge
    edges_to_remove = {}
    for key in hetero_data.keys():
        if 'edge_index' in key:
            edge_data = hetero_data[key]['edge_index']
            total_edges = edge_data.shape[1]
            edges_to_remove_count = int(total_edges * proportion)

            # Seleziona casualmente gli edge da rimuovere
            indices_to_remove = random.sample(range(total_edges), edges_to_remove_count)
            mask = [i for i in range(total_edges) if i not in indices_to_remove]

            # Ritorna un nuovo edge_index con gli edge rimanenti
            edges_to_remove[key] = edge_data[:, mask]

    # Aggiorna la struttura degli edge
    for key, new_edge_index in edges_to_remove.items():
        hetero_data[key]['edge_index'] = new_edge_index

    return hetero_data

In [6]:
def build_nodes_dictionary(KG_data, node_names, val_timestamp, test_timestamp):
    nodes_dict = {}
    seq_number = 0  # Inizializzazione del contatore sequenziale

    for node_name in node_names:
        # Estrazione dei dati del nodo
        node_data = KG_data[node_name]

        # Controllo se il campo 'time' esiste
        if 'time' in node_data:
            timestamps = node_data['time']  # Supponendo che i timestamp siano in un campo 'time'

            for index, time in enumerate(timestamps):
                time = pd.to_datetime(time.item(), unit='s')

                # Determina se il nodo è in train, validation o test
                if time < val_timestamp:
                    category = 'train'
                elif val_timestamp <= time < test_timestamp:
                    category = 'validation'
                else:
                    category = 'test'

                # Creazione della chiave e valore da inserire nel dizionario
                key = f"{node_name}_{index}"
                value = [seq_number, category]

                # Aggiunta al dizionario
                nodes_dict[key] = value
                seq_number += 1  # Incrementa il contatore per ogni nodo analizzato
        else:
            # Se il campo 'time' non esiste
            for index in range(len(node_data)):  # Itera sugli indici dei nodi
                key = f"{node_name}_{index}"
                value = [seq_number, 'all']  # Categoria 'all' se non c'è 'time'

                # Aggiunta al dizionario
                nodes_dict[key] = value
                seq_number += 1  # Incrementa il contatore

    return nodes_dict


In [7]:
def modify_node_name(key):
    # Trova l'ultima posizione di '_' e rimuove la parte numerica
    last_underscore_index = key.rfind('_')

    if last_underscore_index != -1:
        # Parte letterale (es. 'standings')
        modified_name = key[:last_underscore_index]

        # Parte numerica (es. 3)
        node_index = int(''.join(filter(str.isdigit, key[last_underscore_index:])))

        return modified_name, node_index

    else:
        node_name = ''.join(filter(str.isalpha, key))  # Parte letterale (es. 'standings')
        node_index = int(''.join(filter(str.isdigit, key)))  # Parte numerica (es. 5)
        return node_name, node_index

In [8]:
def build_data_file(KG_data, node_names, nodes_dictionary, edges_names):
    node_features = []
    labels = []
    edges = []

    train_mask = []
    val_mask = []
    test_mask = []

    for key in nodes_dictionary:
        # Dividi la chiave in parte letterale e parte numerica
        # node_name = ''.join(filter(str.isalpha, key))  # Parte letterale (es. 'standings')
        # node_index = int(''.join(filter(str.isdigit, key)))  # Parte numerica (es. 5)
        node_name, node_index = modify_node_name(key)

        # Estrai il valore delle features usando i dati di KD_data
        print(node_name)
        feature_value = KG_data[node_name].tf[node_index]
        node_features.append(feature_value)

        # Inserisco anche la label del nodo in labels
        labels.append(node_name)

        # Aggiorno le maschere
        split_value = nodes_dictionary[key][1]
        if split_value == "train":
            train_mask.append(True)
            val_mask.append(False)
            test_mask.append(False)
        elif split_value == "validation":
            train_mask.append(False)
            val_mask.append(True)
            test_mask.append(False)
        elif split_value == "test":
            train_mask.append(False)
            val_mask.append(False)
            test_mask.append(True)
        else:
            train_mask.append(True)
            val_mask.append(True)
            test_mask.append(True)

    # return  node_features, labels, edges, train_mask, val_mask, test_mask

#'''
        # Ora devo inserire gli edges in cui è presente quel nodo
        for edge in edges_names:
            # controllo che quel tipo di nodo è sorgente di un possibile edge
            if node_name == edge[0]:
                # se l'edge ha dentro quel tipo di nodo allora
                edge_index = KG_data[edge].edge_index # retrieve indexes
                # prendo tutti gli indici dei nodi sorgente e dei nodi destinazione
                source_nodes = edge_index[0]  # source indexes
                target_nodes = edge_index[1]  # targer indexes

                source_dict_indices = []
                target_dict_indices = []

                # Creazione delle nuove chiavi
                for source_index in source_nodes:
                    source_key = f"{node_name}_{source_index.item()}"  # Concatenazione con il nome del nodo
                    source_dict_indices.append(nodes_dictionary[source_key][0])

                for target_index in target_nodes:
                    target_key = f"{node_name}_{target_index.item()}"  # Concatenazione
                    target_dict_indices.append(nodes_dictionary[target_key][0])


                # Creazione delle coppie e aggiunta alla lista edges
                for src_index in source_dict_indices:
                    for tgt_index in target_dict_indices:
                        edge_pair = [src_index, tgt_index]
                        if edge_pair not in edges:  # Controllo se la coppia non è già presente
                            edges.append(edge_pair)
        break

    return   node_features, labels, edges, train_mask, val_mask, test_mask
#'''

# F1 Dataset Creation

In [9]:
# We download the f1-dataset
f1_dataset = get_dataset(name="rel-f1", download=True)

# we download the entire database (also the test part)
f1_db = f1_dataset.get_db(upto_test_timestamp = False)
f1_col_to_stype_dict = get_stype_proposal(f1_db)

# Generate graph data
f1_data, f1_col_stats_dict = make_pkey_fkey_graph(
    f1_db,
    col_to_stype_dict = f1_col_to_stype_dict,  # Column types
    text_embedder_cfg = text_embedder_cfg,  # Our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-f1_materialized_cache"
    ),  # Store materialized graph for convenience
)

Loading Database object from /root/.cache/relbench/rel-f1/db...
Done in 0.15 seconds.


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

In [10]:
f1_val_timestep = f1_dataset.val_timestamp
f1_test_timestep = f1_dataset.test_timestamp

print(f"The validation timestep is: {f1_val_timestep}")
print(f"The test timestep is: {f1_test_timestep}")

The validation timestep is: 2005-01-01 00:00:00
The test timestep is: 2010-01-01 00:00:00


In [11]:
f1_data

HeteroData(
  standings={
    tf=TensorFrame([34124, 4]),
    time=[34124],
  },
  drivers={ tf=TensorFrame([857, 6]) },
  results={
    tf=TensorFrame([26080, 11]),
    time=[26080],
  },
  constructor_results={
    tf=TensorFrame([12290, 2]),
    time=[12290],
  },
  circuits={ tf=TensorFrame([77, 7]) },
  qualifying={
    tf=TensorFrame([9815, 3]),
    time=[9815],
  },
  races={
    tf=TensorFrame([1101, 5]),
    time=[1101],
  },
  constructors={ tf=TensorFrame([211, 3]) },
  constructor_standings={
    tf=TensorFrame([13051, 4]),
    time=[13051],
  },
  (standings, f2p_raceId, races)={ edge_index=[2, 34124] },
  (races, rev_f2p_raceId, standings)={ edge_index=[2, 34124] },
  (standings, f2p_driverId, drivers)={ edge_index=[2, 34124] },
  (drivers, rev_f2p_driverId, standings)={ edge_index=[2, 34124] },
  (results, f2p_raceId, races)={ edge_index=[2, 26080] },
  (races, rev_f2p_raceId, results)={ edge_index=[2, 26080] },
  (results, f2p_driverId, drivers)={ edge_index=[2, 26080] 

In [ ]:
updated_data = remove_proportion(f1_data, 0.9)

In [12]:
node_names = ['standings', 'drivers', 'results', 'constructor_results', 'circuits', 'qualifying', 'races', 'constructors', 'constructor_standings']
edges_names = [('constructor_standings', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'constructor_standings'),
                ('constructor_standings', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'constructor_standings'),
                ('standings', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'standings'),
                ('standings', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'standings'),
                ('constructor_results', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'constructor_results'),
                ('constructor_results', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'constructor_results'),
                ('results', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'results'),
                ('results', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'results'),
                ('results', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'results'),
                ('qualifying', 'f2p_raceId', 'races'),
                ('races', 'rev_f2p_raceId', 'qualifying'),
                ('qualifying', 'f2p_driverId', 'drivers'),
                ('drivers', 'rev_f2p_driverId', 'qualifying'),
                ('qualifying', 'f2p_constructorId', 'constructors'),
                ('constructors', 'rev_f2p_constructorId', 'qualifying'),
                ('races', 'f2p_circuitId', 'circuits'),
                ('circuits', 'rev_f2p_circuitId', 'races')]

In [13]:
F1_dict = build_nodes_dictionary(KG_data = f1_data,
                                 node_names = node_names,
                                 val_timestamp = f1_val_timestep,
                                 test_timestamp = f1_test_timestep)

In [14]:
F1_dict

{'standings_0': [0, 'train'],
 'standings_1': [1, 'train'],
 'standings_2': [2, 'train'],
 'standings_3': [3, 'train'],
 'standings_4': [4, 'train'],
 'standings_5': [5, 'train'],
 'standings_6': [6, 'train'],
 'standings_7': [7, 'train'],
 'standings_8': [8, 'train'],
 'standings_9': [9, 'train'],
 'standings_10': [10, 'train'],
 'standings_11': [11, 'train'],
 'standings_12': [12, 'train'],
 'standings_13': [13, 'train'],
 'standings_14': [14, 'train'],
 'standings_15': [15, 'train'],
 'standings_16': [16, 'train'],
 'standings_17': [17, 'train'],
 'standings_18': [18, 'train'],
 'standings_19': [19, 'train'],
 'standings_20': [20, 'train'],
 'standings_21': [21, 'train'],
 'standings_22': [22, 'train'],
 'standings_23': [23, 'train'],
 'standings_24': [24, 'train'],
 'standings_25': [25, 'train'],
 'standings_26': [26, 'train'],
 'standings_27': [27, 'train'],
 'standings_28': [28, 'train'],
 'standings_29': [29, 'train'],
 'standings_30': [30, 'train'],
 'standings_31': [31, 'train

In [ ]:
node_features, labels, edges, train_mask, val_mask, test_mask = build_data_file(KG_data = f1_data,
                                                                                node_names = node_names,
                                                                                nodes_dictionary = F1_dict,
                                                                                edges_names = edges_names)

standings


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-dc3adaa1e589>", line 1, in <cell line: 0>
    node_features, labels, edges, train_mask, val_mask, test_mask = build_data_file(KG_data = f1_data,
                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-70616300896c>", line None, in build_data_file
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Tra